# Build and evaluate predictive models
### What we do here:
   * Data Preparation
   * Build Models
   * Evaluate Model
   * Feature Importance Analysis
   * Use Shap to interpret Model Predictions

### 1.Data Preparation

#### 1.1 Load the Data 

In [2]:
import pandas as pd 
df=pd.read_csv("../data/insurance_cleaned.csv")

C:\Users\UserK\AppData\Local\Temp\ipykernel_3180\677661728.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../data/insurance_cleaned.csv")


In [3]:
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,LossRatio,Month
0,145249,12827,2015-03-01,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,3
1,145249,12827,2015-05-01,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,5
2,145249,12827,2015-07-01,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.0,7
3,145255,12827,2015-05-01,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0.0,5
4,145255,12827,2015-07-01,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.0,7


In [4]:
df.isnull().sum()

UnderwrittenCoverID              0
PolicyID                         0
TransactionMonth                 0
IsVATRegistered                  0
Citizenship                 895210
LegalType                        0
Title                            0
Language                         0
Bank                        145961
AccountType                  40232
MaritalStatus                 8259
Gender                        9536
Country                          0
Province                         0
PostalCode                       0
MainCrestaZone                   0
SubCrestaZone                    0
ItemType                         0
mmcode                           0
VehicleType                    552
RegistrationYear                 0
make                           552
Model                          552
Cylinders                        0
cubiccapacity                    0
kilowatts                        0
bodytype                       552
NumberOfDoors                    0
VehicleIntroDate    

In [5]:
df.dtypes

UnderwrittenCoverID           int64
PolicyID                      int64
TransactionMonth             object
IsVATRegistered                bool
Citizenship                  object
LegalType                    object
Title                        object
Language                     object
Bank                         object
AccountType                  object
MaritalStatus                object
Gender                       object
Country                      object
Province                     object
PostalCode                    int64
MainCrestaZone               object
SubCrestaZone                object
ItemType                     object
mmcode                      float64
VehicleType                  object
RegistrationYear              int64
make                         object
Model                        object
Cylinders                   float64
cubiccapacity               float64
kilowatts                   float64
bodytype                     object
NumberOfDoors               

#### 1.1 Drop columns with highest missing value

In [6]:
# Drop columns with Highest Values
df = df.drop(columns=["Citizenship", "Bank","NewVehicle"])
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,LegalType,Title,Language,AccountType,MaritalStatus,Gender,...,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,LossRatio,Month
0,145249,12827,2015-03-01,True,Close Corporation,Mr,English,Current account,Not specified,Not specified,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,3
1,145249,12827,2015-05-01,True,Close Corporation,Mr,English,Current account,Not specified,Not specified,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,5
2,145249,12827,2015-07-01,True,Close Corporation,Mr,English,Current account,Not specified,Not specified,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.0,7
3,145255,12827,2015-05-01,True,Close Corporation,Mr,English,Current account,Not specified,Not specified,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0.0,5
4,145255,12827,2015-07-01,True,Close Corporation,Mr,English,Current account,Not specified,Not specified,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.0,7


In [7]:
# create targets
df["HasClaim"] = (df["TotalClaims"] > 0).astype(int)
df["Margin"] = df["TotalPremium"] - df["TotalClaims"]



#### 1.3 Drop the rows with missing vehicle Identity

In [8]:
df = df.dropna(subset=["VehicleType", "make", "Model","VehicleIntroDate"])
df.make.isnull().sum()

np.int64(0)

#### 1.4 Impute Catagorical with mode


In [9]:
for col in ["AccountType","MaritalStatus"]:
    df[col]=df[col].fillna(df[col].mode()[0])
   

#### 1.5 Handle gender columns safely

In [10]:
df["Gender"]=df["Gender"].fillna("unknown")

In [11]:
# check Missing Values
df.isnull().sum()

UnderwrittenCoverID         0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
LegalType                   0
Title                       0
Language                    0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
AlarmImmobiliser            0
TrackingDevice              0
SumInsured                  0
TermFrequency               0
CalculatedPremiumPerTerm    0
ExcessSelected              0
CoverCateg

#### 2.Encoding Catagorical Columns

In [12]:
# parse Date columns first
df["TransactionMonth"] = pd.to_datetime(df["TransactionMonth"])
df["VehicleIntroDate"] = pd.to_datetime(df["VehicleIntroDate"])
# create Date based features
df["TransactionYear"] = df["TransactionMonth"].dt.year
df["TransactionMonthNum"] = df["TransactionMonth"].dt.month
df["VehicleAge"] = df["TransactionYear"] - df["VehicleIntroDate"].dt.year
# drop the columns 
df = df.drop(columns=["TransactionMonth", "VehicleIntroDate"])



In [13]:
# label encoding for High-Cardinality Categorical → LABEL ENCODING (or TARGET ENCODING)
label_features = [
    "Province", "PostalCode", "make", "Model",
    "Product", "MainCrestaZone", "SubCrestaZone"
]
label_encoders={}
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in label_features:
    df[col]=le.fit_transform(df[col].astype(str))
    label_encoders[col]=le

In [14]:
# one Hot-encoding for low-Cardinality Categorical → ONE-HOT ENCODING
onehot_features = [
    "Gender", "MaritalStatus", "LegalType", "CoverType",
    "CoverGroup", "Section", "TermFrequency",
    "ExcessSelected", "AlarmImmobiliser", "TrackingDevice"
]
# one Hot encoding
df=pd.get_dummies(
    df,
    columns=onehot_features,
    drop_first=True
)

In [15]:
# check data types before Modeling
df.dtypes.value_counts()

bool       64
int64      12
float64    11
object     10
int32       3
Name: count, dtype: int64

In [16]:
df.select_dtypes(include="object").columns

Index(['Title', 'Language', 'AccountType', 'Country', 'ItemType',
       'VehicleType', 'bodytype', 'CoverCategory', 'StatutoryClass',
       'StatutoryRiskType'],
      dtype='object')

In [17]:
drop_cols=["Title","Language","AccountType","Country","ItemType",
           "VehicleType","bodytype","CoverCategory","StatutoryClass","StatutoryRiskType"] # drop b/c they add noise
df=df.drop(columns=[c for c in drop_cols if c in df.columns ])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999546 entries, 0 to 1000097
Data columns (total 90 columns):
 #   Column                                                                         Non-Null Count   Dtype  
---  ------                                                                         --------------   -----  
 0   UnderwrittenCoverID                                                            999546 non-null  int64  
 1   PolicyID                                                                       999546 non-null  int64  
 2   IsVATRegistered                                                                999546 non-null  bool   
 3   Province                                                                       999546 non-null  int64  
 4   PostalCode                                                                     999546 non-null  int64  
 5   MainCrestaZone                                                                 999546 non-null  int64  
 6   SubCrestaZone   

In [19]:
bool_cols = df.select_dtypes(include="bool").columns
df[bool_cols] = df[bool_cols].astype(int)


In [20]:
df.dtypes.value_counts()

int64      76
float64    11
int32       3
Name: count, dtype: int64

### 3.Separate Data 

#### 3.1 Claim Probability Model(Classification)


In [21]:
# take Data for classisfication and x and y split
df["HasClaim"] = (df["TotalClaims"] > 0).astype(int)

X_prob = df.drop(columns=["TotalClaims", "HasClaim"])
y_prob = df["HasClaim"]


#### 3.2 Claim Saverity Model(Regression)

In [22]:
# take Data for regression model and split the data into x and y 
severity_df = df[df["TotalClaims"] > 0].copy()
X_sev = severity_df.drop(columns=["TotalClaims"])
y_sev = severity_df["TotalClaims"]

### 4.Train-Test Split

#### 4.1 split for Classification

In [23]:
from sklearn.model_selection import train_test_split
x_train_p,x_test_p,y_train_p,y_test_p=train_test_split(X_prob,y_prob,
                                               test_size=0.2,random_state=42,stratify=y_prob)

#### 4.2 split for classification



In [24]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_sev, y_sev,
    test_size=0.2,
    random_state=42
)


### 5.Models

#### 5.1 Logistic Regression(Classification)

In [25]:
from sklearn.linear_model import LogisticRegression # import the model
from sklearn.metrics import classification_report,roc_auc_score
# create the model object
log_reg=LogisticRegression(random_state=42)
log_reg.fit(x_train_p,y_train_p) # train on the training data
y_pred = log_reg.predict(x_test_p)
y_proba = log_reg.predict_proba(x_test_p)[:, 1]
accuracy = log_reg.score(x_test_p, y_test_p)
print("accuracy",accuracy)
print(classification_report(y_test_p, y_pred))
print("ROC-AUC:", roc_auc_score(y_test_p, y_proba))


d:\FInalProject\End-to-End-Insurance-Risk-Analytics\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


accuracy 0.9998999549797409
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199355
           1       1.00      0.97      0.98       555

    accuracy                           1.00    199910
   macro avg       1.00      0.98      0.99    199910
weighted avg       1.00      1.00      1.00    199910

ROC-AUC: 0.9951572560245531


#### 5.2 Random Forest 

In [26]:
from sklearn.ensemble import RandomForestClassifier
# create the model object
rf=RandomForestClassifier(n_estimators=2000,
                          n_jobs=-1,
                          random_state=42)
# train on the training data 
rf.fit(x_train_p,y_train_p)
y_pred = log_reg.predict(x_test_p)
y_proba = log_reg.predict_proba(x_test_p)[:, 1]
print("accuracy",accuracy)
print(classification_report(y_test_p, y_pred))
print("ROC-AUC:", roc_auc_score(y_test_p, y_proba))


accuracy 0.9998999549797409
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199355
           1       1.00      0.97      0.98       555

    accuracy                           1.00    199910
   macro avg       1.00      0.98      0.99    199910
weighted avg       1.00      1.00      1.00    199910

ROC-AUC: 0.9951572560245531


####  5.3 Severity Regression (RMSE + R²)

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rf_reg = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

rf_reg.fit(X_train_s, y_train_s)

y_pred_s = rf_reg.predict(X_test_s)

rmse = np.sqrt(mean_squared_error(y_test_s, y_pred_s))
r2 = r2_score(y_test_s, y_pred_s)

print("RMSE:", rmse)
print("R²:", r2)


RMSE: 624.7025862329986
R²: 0.9997259692139239


### 6 Feature Importance 
 

In [28]:

# Knowing which feature was the most important one
importances = pd.Series(
    rf.feature_importances_,
    index=x_train_p.columns
).sort_values(ascending=False)

importances.head(15)


Margin                      0.519826
LossRatio                   0.443930
TotalPremium                0.004582
UnderwrittenCoverID         0.004143
PolicyID                    0.003419
TransactionMonthNum         0.003331
Month                       0.003321
CalculatedPremiumPerTerm    0.002161
PostalCode                  0.001745
SumInsured                  0.001269
RegistrationYear            0.001156
ExcessSelected_No excess    0.000965
Model                       0.000886
VehicleAge                  0.000863
SubCrestaZone               0.000823
dtype: float64

### 7.Using Lime For Interpretation

In [29]:
import shap # importing


d:\FInalProject\End-to-End-Insurance-Risk-Analytics\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 7.1 Lime for Claim Probability Model (Random Forest Classifier)

In [32]:
from lime.lime_tabular import LimeTabularExplainer
explainer = LimeTabularExplainer(
    training_data=x_train_p.values,
    feature_names=x_train_p.columns,
    class_names=["No Claim", "Claim"],
    mode="classification"
)
i = 5  # choose any policy index

exp = explainer.explain_instance(
    x_test_p.iloc[i].values,
    rf.predict_proba,
    num_features=10
)
exp.as_list()


d:\FInalProject\End-to-End-Insurance-Risk-Analytics\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


[('LossRatio <= 0.00', -0.2676816902394819),
 ('2.16 < Margin <= 21.93', -0.2648328443322542),
 ('CoverGroup_Fire,Theft and Third Party <= 0.00', 0.16605975035292136),
 ('CoverGroup_Standalone passenger liability <= 0.00', -0.15694342416710455),
 ('Section_Third party or third party, fire and theft only <= 0.00',
  0.15361536097250356),
 ('CoverType_Deposit Cover <= 0.00', -0.13035804787314076),
 ('CoverType_Trailer <= 0.00', 0.1257582936238978),
 ('CoverType_Standalone passenger liability <= 0.00', 0.12417384451742118),
 ('CoverType_Cash Takings <= 0.00', 0.11377400509310281),
 ('CoverGroup_Cash Takings <= 0.00', 0.10245481631654815)]

#### 7.2 LIME for Claim Severity Model (Regression)


In [35]:
explainer_reg = LimeTabularExplainer(
    training_data=X_train_s.values,
    feature_names=X_train_s.columns,
    mode="regression"
)
i = 8

exp_reg = explainer_reg.explain_instance(
    X_test_s.iloc[i].values,
    rf_reg.predict,
    num_features=10
)
exp_reg.as_list()



d:\FInalProject\End-to-End-Insurance-Risk-Analytics\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


[('Margin > -1664.94', -36048.26518998906),
 ('CoverType_Roadside Assistance <= 0.00', -27553.912735121972),
 ('LegalType_Public company <= 0.00', 16557.93404311646),
 ('CoverGroup_Credit Protection <= 0.00', -15314.176167555826),
 ('ExcessSelected_Mobility - Taxi with value more than R100 000 - R7 500 <= 0.00',
  14435.276703998334),
 ('CoverGroup_Motor Comprehensive <= 0.00', 10846.924553005307),
 ('CoverType_Factory Fitted Sound and Other Electronic Equipment <= 0.00',
  9780.977542379156),
 ('CoverGroup_Deposit Cover <= 0.00', 9508.89443013191),
 ('CoverType_Emergency Charges <= 0.00', 8337.952186719152),
 ('MaritalStatus_Single <= 0.00', 7576.382090982715)]